# Recitation 1 - Formulations
In today's recitation, we will formulation logical variables for scheduling

# amoon summary

## 1. Polyhedral theory review
vector in convex cone or there exist hyperplane that separates vector from convex cone
either \exist x >=0 \in \R^n s.t. {x: Ax= b} or \exist p \in R^m s.t. {p'b <0 <= p'A}
either \exist x \in \R^n s.t. {x: Ax <=b} or \exist p \in R^m s.t. {p'b < 0 = p'A}

C(p) = {0}
if extreme ray of polyhedron is 0, proj_x(P) = R^n

farkas + NA technique: translate primal's inf to dual's ubd or inf (but has feas) hence ubd
WV thm: every polyhedron can be finitely generated with conic combination of ext ray

linear: $\Sigma \lambda_i x_i$
affine: $\Sigma \lambda_i x_i, \lambda_i \geq 0$
conic: $\lambda_i \geq 0$
convex: $\Sigma \lambda_i x_i, \Sigma \lambda_i = 1$ (line segment)

Projection P: {(x,y) \in R^n \cross R^p: Ax + By < b}
proj_x(P) = {x \in R^n, }
primal is inf -> dual is inf or ubd but as p = 0 is feasible as we set the primal's obj function as 0. there exists p'(b-Ax) <0, pB = 0, p>0

## 2. Course schedule
### Problem Statement

You are planning the course schedule at the Sloan School for next Semester. To simplify the
problem, you make the following assumptions.

On the operations side, Sloan offers 30 courses. Most are offered only once, but some are repeated
in several sessions (for a total of 55 sessions). Each session has the same schedule on Mondays/Tuesdays and Wednesdays/Thursdays; hence, we consider the problem over two days (e.g., Mondays
and Tuesdays). Each day has six 90-minute time slots (two morning slots, three afternoon slots
and one evening slot). Sloan has access to 30 classrooms, each with limited seating capacity.

On the demand side, we consider five groups of students: (i) first-year MBA students, (ii) first-year
MBAN students; (iii) first-year ORC SM students; (iv) first-year ORC PhD students; and (v) other
students. We divide students into smaller cohorts, for two reasons. First, due to the large size of
the program, first-year MBA students are divided into “oceans” of 60 students each, who take core
courses together. Second, other students have different interests in electives, and cohorts designate
smaller groups with similar interests (these can be calibrated from historical course selections, for
instance). Ultimately, we consider 57 cohorts: 6 MBA cohorts of 60 students each, 7 MBAN cohorts
of 10 students each, 2 ORC SM cohorts of 5 students each, 2 ORC PhD cohorts of 10 students
each, and 40 cohorts of 12 students each, capturing students in other programs.

You define the following inputs, decision variables and constraints:

#### Sets

$$\begin{aligned}
    \mathcal{S}    &:   \ \text{set of student cohorts} \\
    \mathcal{C}    &:   \ \text{set of courses}         \\
    \mathcal{M}_s  &:   \ \text{subset of mandatory courses for cohort $s\in\mathcal{S}$} \\
    \mathcal{J}    &:   \ \text{set of course sessions} \\
    \mathcal{J}_c  &:   \ \text{subset of course sessions belonging to course $c\in\mathcal{C}$} \\
    \mathcal{D}    &:   \ \text{set of days} \\
    \mathcal{T}    &:   \ \text{set of time blocks within a day} \\
    \mathcal{R}    &:   \ \text{set of classrooms}  \end{aligned}$$

#### Parameters

$$\begin{aligned}
    N_s            &:   \ \text{number of students in cohort $s\in\mathcal{S}$} \\
    Q_r            &:   \ \text{capacity of room $r\in\mathcal{R}$} \\
    U_c            &:   \ \text{number of units of course $c\in\mathcal{C}$} \\
    m_s            &:   \ \text{minimum number of units of cohort $s\in\mathcal{S}$} \\
    M_s            &:   \ \text{maximum number of units of cohort $s\in\mathcal{S}$} \end{aligned}$$

#### Decision variables

$$\begin{aligned}
        x_{jdtr}&=  \begin{cases}
                    1   &   \text{if session $j\in\mathcal{J}$ is offered on day $d\in\mathcal{D}$ at time $t\in\mathcal{T}$ in room $r\in\mathcal{R}$}  \\
                    0   &   \text{otherwise}    \\
                    \end{cases}    \\
        z_{sjdt}&=  \begin{cases}
                    1   &   \text{if student group $s\in\mathcal{S}$ takes session $j\in\mathcal{J}$ on day $d\in\mathcal{D}$ at time $t\in\mathcal{T}$}  \\
                    0   &   \text{otherwise}    \\
                    \end{cases}    
    \end{aligned}$$

# Load the Data

We first need to bring in the problem data. 

In [1]:
import Pkg
Pkg.update("JuMP")
Pkg.build("Gurobi")
Pkg.add("Gurobi")

    Updating registry at `~/.julia/registries/General.toml`
   Installed GR_jll ────────────────── v0.71.8+0
   Installed HypergeometricFunctions ─ v0.3.15
   Installed StatsFuns ─────────────── v1.3.0
   Installed FFTW ──────────────────── v1.6.0
   Installed MutableArithmetics ────── v1.2.3
   Installed PDMats ────────────────── v0.11.17
   Installed StaticArrays ──────────── v1.5.21
   Installed IntelOpenMP_jll ───────── v2023.1.0+0
   Installed InvertedIndices ───────── v1.3.0
   Installed OpenSSL ───────────────── v1.3.5
   Installed IrrationalConstants ───── v0.2.2
   Installed JSON ──────────────────── v0.21.4
   Installed SentinelArrays ────────── v1.3.18
   Installed NaNMath ───────────────── v1.0.2
   Installed SpecialFunctions ──────── v2.2.0
   Installed TranscodingStreams ────── v0.9.12
   Installed StatsAPI ──────────────── v1.6.0
   Installed Compat ────────────────── v4.6.1
   Installed MathOptInterface ──────── v1.15.0
   Installed DiffRules ─────────────── v1.13.0
   

     Deleted 6 package installations (5.663 MiB)
    Building Gurobi → `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/82a44a86f4dc4fa4510c9d49b0a74d3d73914d5c/build.log`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [2]:
using JuMP, Gurobi, Distances, Distributions, DataFrames, CSV

If you have not previously installed some or all of the packages above, do so with the code below (it may take a minute to install).

In [3]:
using Pkg
Pkg.add("Distances")
Pkg.add("Distributions")
Pkg.add("DataFrames")
Pkg.add("CSV")
using JuMP, Gurobi, Distances, Distributions, DataFrames, CSV

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


Now we will read in the data that was included in the zip file for today's recitation.

In [4]:
sessions = CSV.read("data/sessions.csv", DataFrame);
courses = CSV.read("data/courses.csv", DataFrame);
rooms = CSV.read("data/rooms.csv", DataFrame);
preferences = CSV.read("data/preferences.csv", DataFrame);
cohorts = CSV.read("data/cohorts.csv", DataFrame);

In [5]:
sessions

Row,session,course
,Int64,Int64
1,1,1
2,2,1
3,3,1
4,4,1
5,5,1
6,6,1
7,7,2
8,8,2
9,9,2


Let's look at the data!

In [3]:
courses

Row,course,units,comments
,Int64,Int64,String?
1,1,9,mandatory for cohort 1 through 6 (MBA)
2,2,9,mandatory for cohort 1 through 6 (MBA)
3,3,9,mandatory for cohort 1 through 6 (MBA)
4,4,9,mandatory for cohort 1 through 6 (MBA)
5,5,9,mandatory for cohort 1 through 6 (MBA)
6,6,12,mandatory for cohort 7 through 13 (MBAN)
7,7,12,mandatory for cohort 7 through 13 (MBAN)
8,8,9,mandatory for cohort 7 through 13 (MBAN)
9,9,6,mandatory for cohort 7 through 13 (MBAN)


In [4]:
@show nrow(sessions)
@show nrow(courses)
@show nrow(rooms)
@show nrow(preferences)
@show nrow(cohorts)

first(sessions, 5)

nrow(sessions) = 55
nrow(courses) = 30
nrow(rooms) = 20
nrow(preferences) = 57
nrow(cohorts) = 57


Row,session,course
,Int64,Int64
1,1,1
2,2,1
3,3,1
4,4,1
5,5,1


Convert the data to matrices for easy use in the optimization model

In [5]:
sessions = Matrix(sessions);
courses = Matrix(courses);
rooms = Matrix(rooms);
preferences = Matrix(preferences);
cohorts = Matrix(cohorts);

In [6]:
sessions

Row,session,course
,Int64,Int64
1,1,1
2,2,1
3,3,1
4,4,1
5,5,1
6,6,1
7,7,2
8,8,2
9,9,2


# Optimization Model

We'll first bring in some helpful sets and notation:

In [6]:
J = size(sessions,1);
C = size(courses,1);
R = size(rooms,1);
S = size(cohorts,1);
D = 2;
T = 6;
Q = rooms[:,2];
units = courses[:,2];
N = cohorts[:,2];
min_units = cohorts[:,3];
max_units = cohorts[:,4];

#Manually loading the mandatory courses, 
#using the third column of the courses matrix - where does 6, 14, 16 comes from?
mandatory = zeros(S,C);
mandatory[1:6,1:5] .= 1;
mandatory[7:13,6:9] .= 1;
mandatory[14:15,10:11] .= 1;
mandatory[16:17,12:13] .= 1;

We'll now create the model using Gurobi optimizer and setting a time limit of 60 seconds

In [7]:
model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "TimeLimit", 60);

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-11
Set parameter TimeLimit to value 60


## Decision Variables

In [8]:
@variable(model, z[1:S,1:J,1:D,1:T], Bin)
@variable(model, x[1:J,1:D,1:T,1:R], Bin) ; 

## Constraints

Let's write some!

#### 1. Every session $j$ is assigned to exactly one room $r$, one day $d$, and one time $t$

$$\begin{aligned}
                &   \quad   \sum_{r\in\mathcal{R}}\sum_{d\in\mathcal{D}}\sum_{t\in\mathcal{T}}x_{jdtr}=1,                &  \forall j\in\mathcal{J}  \\
\end{aligned}$$

In [9]:
@constraint(model, sessionassignment[j in 1:J], sum(x[j,d,t,r] for r in 1:R, d in 1:D, t in 1:T) == 1); #why name if not used after?

#### 2. Each student group $s$ must be assigned a session of all of their required courses
$$\begin{aligned}
             &   \quad   \sum_{d\in\mathcal{D}}\sum_{t\in\mathcal{T}}\sum_{j\in\mathcal{J}_c}z_{sjdt}\geq 1,                &  \forall s\in\mathcal{S},\ \forall c\in\mathcal{M}_s   \\
\end{aligned}$$

In [12]:
for s in 1:S
    mandatory_s = findall(x -> x==1, mandatory[s, :]);
    for c in mandatory_s
        @constraint(
            model, 
            sum(
                z[s,j,d,t] 
                for d in 1:D, 
                    t in 1:T, 
                    j in sessions[findall(x -> x==c,sessions[:,2]),1]
            ) >= 1
        )
    end
end

#### 3. *What is this constraint?*
$$\begin{aligned}
                &   \quad   \sum_{j\in\mathcal{J}}z_{sjdt}\leq 1,                &  \forall s\in\mathcal{S},\ \forall d\in\mathcal{D},\ \forall t\in\mathcal{T}   \\
\end{aligned}$$

Notation reminder: $z_{sjdt}$ assigns student group $s$ to take session $j$ on day $d$ at time $t$

In [13]:
@constraint(
    model, 
    mysteryconstraint[s in 1:S, d in 1:D, t in 1:T],
    sum(z[s,j,d,t] for j in 1:J) <= 1
);

ErrorException: An object of name mysteryconstraint is already attached to this model. If this
    is intended, consider using the anonymous construction syntax, e.g.,
    `x = @variable(model, [1:N], ...)` where the name of the object does
    not appear inside the macro.

    Alternatively, use `unregister(model, :mysteryconstraint)` to first unregister
    the existing name from the model. Note that this will not delete the
    object; it will just remove the reference at `model[:mysteryconstraint]`.


<span style="color:red">**ANSWER:**</span> **Each student group $s$ can be assigned to at most one session during
each time slot (i.e. at time $t$ on day $d$).**


#### 4. *What is this constraint?*
$$\begin{aligned}
                &   \quad   \sum_{d\in\mathcal{D}}\sum_{t\in\mathcal{T}}\sum_{j\in\mathcal{J}_c}z_{sjdt}\leq 1,                &  \forall s\in\mathcal{S},\ \forall c\in\mathcal{C}    \\
\end{aligned}$$

Notation reminders: 
 - $z_{sjdt}$ assigns student group $s$ to take session $j$ on day $d$ at time $t$
 - $\mathcal{J}_c$ is the set of sessions that correspond to course $c$

In [14]:
for c in 1:C
    @constraint(
        model, 
        [s in 1:S], #can't use c in 1:C here? 
        sum(
            z[s,j,d,t] 
            for d in 1:D, 
                t in 1:T, 
                j in sessions[findall(x -> x==c, sessions[:,2]),1]
        ) <= 1
    );
end

<span style="color:red">**ANSWER:**</span> **Each student group $s$ should only be assigned at most one session of
any course (cannot take two sessions of the same course).** 

#### 5.  *What is this constraint?*


$$\begin{aligned}
                &   \quad   \sum_{j\in\mathcal{J}}x_{jdtr}\leq 1,                
                &  \forall r\in\mathcal{R},\ \forall d\in\mathcal{D},\ \forall t\in\mathcal{T}   \\
\end{aligned}$$

Notation reminder: $x_{jdtr}$ assigns session $j$ to room $r$ on day $d$ at time $t$

In [15]:
@constraint( #+ confused about the order of for loop
    model, 
    mysteryconstraint2[r in 1:R, d in 1:D, t in 1:T], 
    sum(x[j,d,t,r] for j in 1:J) <= 1
);

<span style="color:red">**ANSWER:**</span>  **For every room $r$, at most one session can be assigned during each time $t$ and day $d$.** 

#### 6. / 7. *What are these constraints?*

$$\begin{aligned}
                \sum_{d\in\mathcal{D}} \sum_{t\in\mathcal{T}} \sum_{c\in\mathcal{C}} \sum_{j\in\mathcal{J}_c}  U_c z_{sjdt} \leq M_s \quad \forall s \in \mathcal{S} \\
                \sum_{d\in\mathcal{D}} \sum_{t\in\mathcal{T}} \sum_{c\in\mathcal{C}} \sum_{j\in\mathcal{J}_c}  U_c z_{sjdt} \geq m_s \quad \forall s \in \mathcal{S} \\
\end{aligned}$$

Notation reminders: 
 - $U_c$ is the number of units for course $c$
 - $m_s$ and $M_s$ are the minimum and maximum number of units taken for student group $s$
 - $\mathcal{J}_c$ is the set of sessions that correspond to course $c$

In [16]:
@constraint(
    model, 
    mysteryconstraint3[s in 1:S],
    sum(
        units[c] * z[s,j,d,t] 
        for c in 1:C, 
            d in 1:D, 
            t in 1:T, 
            j in sessions[findall(x -> x==c,sessions[:,2]),1]
    ) <= max_units[s]
);

@constraint(
    model, 
    mysteryconstraint4[s in 1:S],
    sum(
        units[c] * z[s,j,d,t] 
        for c in 1:C, 
            d in 1:D, 
            t in 1:T, 
            j in sessions[findall(x -> x==c,sessions[:,2]),1]
    ) >= min_units[s]
);

<span style="color:red">**ANSWER:**</span> 

**6. Each student cohort $s \in \mathcal{S}$ must take no more than their maximum numbers of units $M_s$.**

**7. Each  student cohort $s \in \mathcal{S}$ must take no fewer than their minimum numbers of units $m_s$.** 

### We now have the following constraints:

1. Every session $j$ is assigned to exactly one room $r$, one day $d$, and one time $t$
2. Each student group $s$ must be assigned a session of all of their required courses
3. Each student group $s$ can be assigned to at most one session during each time slot (i.e. at time $t$ on day $d$)
4. Each student group $s$ should only be assigned at most one session of any course (cannot take two sessions of the same course)
5. For every room $r$, at most one session can be assigned during each time $t$ and day $d$
6. Each student cohort $s \in \mathcal{S}$ must take no more than their maximum numbers of units $M_s$.
7. Each  student cohort $s \in \mathcal{S}$ must take no fewer than their minimum numbers of units $m_s$.

### What's missing? 

<span style="color:red">**ANSWER:**</span> **Linking constraints between $x$ and $z$**

#### 7. Student group $s$ can only be assigned to session $j$ on day $d$ at time $t$ if session $j$ has been assigned to a room during that time slot.

*Try writing this constraint yourself, first on paper then in code below*



Notation reminder: 
- $x_{jdtr}$ assigns session $j$ to room $r$ on day $d$ at time $t$
- $z_{sjdt}$ assigns student group $s$ to take session $j$ on day $d$ at time $t$

In [17]:
# ADD CONSTRAINT CODE HERE
@constraint(
    model, 
    xzlink[s in 1:S, j in 1:J, d in 1:D, t in 1:T], 
    z[s,j,d,t] <= sum(x[j,d,t,r] for r in 1:R)
);

<span style="color:red">**ANSWER:**</span> 
$$\begin{aligned}
                &   \quad   z_{sjdt}\leq \sum_{r\in\mathcal{R}}x_{jdtr},               &  \forall s\in\mathcal{S},\ \forall j\in\mathcal{J},\ \forall d\in\mathcal{D},\ \forall t\in\mathcal{T} \\
\end{aligned}$$

#### 8.  The number of students assigned to take session $j$ must not exceed the capacity $Q_r$ of room $r$ to which session $j$ has been assigned.

*Try writing this constraint yourself, first on paper then in code below*



Notation reminder: 
- $x_{jdtr}$ assigns session $j$ to room $r$ on day $d$ at time $t$
- $z_{sjdt}$ assigns student group $s$ to take session $j$ on day $d$ at time $t$
- $Q_{r}$ is the capacity of room $r$ (in code: Q[r])
- $N_{s}$ is the size of student group $s$ (in code: N[s])

<span style="color:red">**ANSWER:**</span> 
$$\begin{aligned}
               &   \quad   \sum_{s\in\mathcal{S}}N_sz_{sjdt}\leq \sum_{r\in\mathcal{R}}Q_rx_{jdtr},                &  \forall j\in\mathcal{J},\ \forall d\in\mathcal{D},\ \forall t\in\mathcal{T}    \\
\end{aligned}$$

In [18]:
# ADD CONSTRAINT CODE HERE
@constraint(
    model, 
    roomcapacity[j in 1:J, d in 1:D, t in 1:T], 
    sum(N[s] * z[s,j,d,t] for s in 1:S) <= sum(Q[r] * x[j,d,t,r] for r in 1:R)
);

## Objective

The objective of the problem is to create many course options but also to prioritize attractive ones. Denote by $\mathcal{P}_s$ the subset of courses that are "preferred" by cohort $s\in\mathcal{S}$. Formulate the objective of maximizing, first, the total number of "preferred" courses taken across the university, and, second, the number of "non-preferred" courses. You can weight the number of "non-preferred" courses by $\lambda=0.1$. For instance, if cohort $s\in\mathcal{S}$ takes 3 "preferred" courses and 2 "non-preferred" ones, it contributes $3.2N_s$ to the objective.

$$\begin{aligned}
                \max &  \sum_{d\in\mathcal{D}} \sum_{t\in\mathcal{T}} \sum_{s\in\mathcal{S}} \sum_{c\in\mathcal{P}_s} \sum_{j\in\mathcal{J}_c}  N_s z_{sjdt} + 0.1 \sum_{d\in\mathcal{D}} \sum_{t\in\mathcal{T}} \sum_{s\in\mathcal{S}} \sum_{c\in\mathcal{C} \setminus \mathcal{P}_s} \sum_{j\in\mathcal{J}_c}  N_s z_{sjdt} \\
\end{aligned}$$



In [19]:
@objective(
    model, 
    Max,
    sum(
        N[s] * z[s,j,d,t]
        for s in 1:S,
            c in findall(x -> x==1,preferences[s,:]),
            j in sessions[findall(x -> x==c,sessions[:,2]),1],
            d in 1:D,
            t in 1:T
    )
    + 0.1 * sum(N[s]*z[s,j,d,t]
        for s in 1:S,
            c in findall(x -> x==0,preferences[s,:]),
            j in sessions[findall(x -> x==c,sessions[:,2]),1],
            d in 1:D,
            t in 1:T
    )
);

# Solve!

We'll now solve the model and look at the objective value.

In [20]:
optimize!(model)

Set parameter TimeLimit to value 60
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 41215 rows, 50820 columns and 1022904 nonzeros
Model fingerprint: 0xc165639b
Variable types: 0 continuous, 50820 integer (50820 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [5e-01, 6e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+01]
Presolve removed 1823 rows and 1296 columns
Presolve time: 1.51s
Presolved: 39392 rows, 49524 columns, 971334 nonzeros
Variable types: 0 continuous, 49524 integer (49524 binary)
Deterministic concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Root barrier log...

Ordering time: 0.01s

Barrier statistics:
 AA' NZ     : 1.736e+05
 Factor NZ  : 6.878e+05 (roughly 10 MB of memory)
 Factor Ops : 1.383e+08 (less than 1 second per iteration)
 T

#### Breakdown of the output:

Usually Gurobi starts with solves the LP relaxation and reports back:

`Root relaxation: objective 5.853400e+03, 107976 iterations, 188.45 seconds`

Now it explores the branch-and-bound tree, and updates us as it goes along. Let's look at just the first line:

    Nodes    |    Current Node    |     Objective Bounds      |     Work         
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 5853.40000    0   61 2974.80000 5853.40000  96.8%     -  211s

We see that the information is broken down into four main columns:

**Nodes**: 
- Global node information  
- How many branch-and-bound nodes have we looked at  
- How many do we have in our queue  

**Current Node**
- Objective
- Depth in the tree
- Number of noninteger variables in the solution

**Objective Bounds**
- Best incumbent solution
- Bound at the node (by solving the relaxation)
- The gap between the two

**Work**
- Sverage simplex iterations per node
- Total elapsed time

As Gurobi continues to work on this problem, the log looks like this:

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 5853.40000    0   61 2974.80000 5853.40000  96.8%     -  211s  
H    0     0                    5774.4000000 5853.40000  1.37%     -  211s                                                     
H    0     0                    5783.0000000 5853.40000  1.22%     -  212s                                                     
     0     0 5853.40000    0  941 5783.00000 5853.40000  1.22%     -  220s  
     0     0 5853.40000    0 1089 5783.00000 5853.40000  1.22%     -  251s  
     0     0 5853.40000    0  162 5783.00000 5853.40000  1.22%     -  275s  
     0     0 5853.40000    0  555 5783.00000 5853.40000  1.22%     -  280s  
     0     0 5853.40000    0  294 5783.00000 5853.40000  1.22%     -  300s  
     0     0 5853.40000    0  871 5783.00000 5853.40000  1.22%     -  316s  
     0     0 5853.40000    0  321 5783.00000 5853.40000  1.22%     -  338s  
     0     0 5853.40000    0  727 5783.00000 5853.40000  1.22%     -  344s  
     0     0 5853.40000    0  139 5783.00000 5853.40000  1.22%     -  357s  
     0     0 5853.40000    0  128 5783.00000 5853.40000  1.22%     -  362s  
     0     2 5853.40000    0  128 5783.00000 5853.40000  1.22%     -  388s  
     1     4 5853.40000    1  156 5783.00000 5853.40000  1.22%  6362  408s  
     3     8 5853.40000    2  587 5783.00000 5853.40000  1.22%  7207  473s  
     7    14 5853.40000    3  515 5783.00000 5853.40000  1.22%  7150  525s  
    13    20 5853.40000    4  680 5783.00000 5853.40000  1.22%  7047  559s  
    19    26 5853.40000    5  680 5783.00000 5853.40000  1.22%  7396  575s  
    31    38 5853.40000    6  557 5783.00000 5853.40000  1.22%  5211  600s  


- Any time there is an 'H' in the first column, Gurobi has used a heuristic to get a new feasible solution. In other logs, you'll sometimes see a '\*' as the first line, indicating a feasible solution was found by branching.
- The first several lines are all exploring the root node using cutting planes
- The last iterations begin exploring the branch and bound tree, trying to find better solutions by improving the upper and lower bounds

In [19]:
objective_value(model)

4746.800001306979

# Look at the solution

We'll look at the schedule for a few groups of students.

In [ ]:
s = 1

println("======= SCHEDULE FOR GROUP $s =======")
for d in 1:D, t in 1:T, j in 1:J
    if value(z[s,j,d,t]) > 0.001
        println("Session $j on day $d at time $t = ", value(z[s,j,d,t]))
    end
end

# Wrap up: Decomposing the problem

To increase the model's scalability and reduce computational times, we want to propose a **decomposition heuristic** that breaks
down the problem into two (or more) smaller sub-problems. 
- What is attractive about your approach?
- Are you guaranteed to find an optimal solution of the overall problem? 
- Are you guaranteed to find a feasible solution? 

*Take 5 minutes to think about how you would break up the problem into smaller decisions then "glue" the pieces together*

<span style="color:red">**POSSIBLE ANSWERS:**</span> 

- One option is to treat the decisions one at a time. In a first step, we optimize course timetabling, ignoring classroom assignments. In a second step, we optimize the assignment of courses to classrooms, treating the course schedule as fixed. This approach mirrors the sequential decision-making processes in place in practice. It can perform relatively well if room capacities are not the main constraints. It is obviously not guaranteed to find the optimal solution, because the first step ignores the impact of timetabling on classroom assignment. It is not even guaranteed to find a feasible solution because the timetabling outcome can lead to too many courses, or too large sessions, being scheduled at the same time (note that this does not occur if some sessions are allowed to move online, which is beyond the scope of this problem). One way to restore feasibility is to limit the number of students taking courses at the same time and the number of simultaneous sessions in the first-step timetabling problem.
- Another option is to treat one student cohort, or a set of cohorts, at a time. For instance, we could first schedule the core courses of first-year MBA students. Then, we fix the schedule of these courses, adjust the inputs accordingly (e.g., classroom availability, remaining classroom capacity) and optimize MBAN classes. We proceed until all cohorts have been considered. This solution can perform well if it starts with the larger cohorts, which exhibits less flexibility and thus impose the strongest constraints into the problem. This solution approach is again not guaranteed to find the optimal solution or even a feasible solution. Moreover, it can result in inequitable outcomes for the cohorts that are treated last.